<a href="https://colab.research.google.com/github/wagnermoraesjr/Etapa_DataPrep/blob/main/Notebook_DataPrep_Problema_Churn_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Preparation (DataPrep)**

A preparação de dados é uma etapa crítica e fundamental que envolve o processamento, limpeza e transformação de dados brutos em um formato adequado para análise e modelagem. A qualidade dos dados é crucial para o sucesso do projeto, e a dataprep aborda questões como valores ausentes, duplicatas e outliers, assegurando a consistência e confiabilidade dos dados. Além disso, a compatibilidade dos dados é alcançada por meio da padronização de formatos e esquemas, permitindo a integração de informações provenientes de diversas fontes.<br><br>


**Objetivo**

Meu objetivo aqui é colocar em prática o aprendizado sobre o processo de preparação dos dados, para que estes dados estejam prontos para serem utilizados no treinamento dos modelos de Machine Learning. Será feito um DataPrep básico, menos complexo, que será suficiente para esse meu objetivo.<br><br>

**Ferramentas**

Nesse caso, como já tenho a minha ABT pronta, com as variáveis explicativas criadas na etapa anterior, vou trabalhar principalmente com o **Python** e a biblioteca **pandas**. Também irei utilizar outras bibliotecas para me auxiliar no processo como a **scikit-learn** para tratamento e a **pickle** para serializar e desserializar objetos Python.<br><br>

**Problema de Negócio**

Para esse trabalho vou usar dados fictícios que simulam as operações de uma empresa que gostaria de avaliar a perda de clientes (Churn).<br><br>

**Base de Dados**

- **ABT de Churn:** Base com informações cadastrais de 1.000 pessoas, já consolidada com as variáveis explicativas criadas na etapa anterior. O arquivo está em Parquet, principalmente pelo fato de ser um formato eficiência no armazenamento e processamento de dados, porém iremos salvar as ABT de treino e teste, tratadas, em formato CSV.

<br>

##**Importação das bibliotecas necessárias**

In [ ]:
# Importando as bibliotecas Pandas e Numpy.
import pandas as pd
import numpy as np

# Importando train_test_split da biblioteca Sklearn.
from sklearn.model_selection import train_test_split

# Importanto o pickle para serialização.
import pickle

# Importando as classes para tratamento de variáveis categóricas.
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# Importando as classes para tratamento de variáveis numéricas.
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

<br>

## **Leitura e visualização da nossa ABT**

In [ ]:
# Lendo diretório Parquet.
abt_churn_00 = pd.read_parquet('/content/drive/MyDrive/abt_churn', engine='pyarrow')
abt_churn_00.shape

(1000, 99)

In [ ]:
abt_churn_00.head()

,ID_CLIENTE,IDADE,NM_GENERO,QT_DIAS_DESDE_INSCR,FL_USOU_SUPORTE,PLANO,CHURN,UTL_DATA_COMPRA,DIAS_DESDE_ULT_TRANS,QTD_DIAS_DESDE_PRIM_COMPRA,...,VL_RAZ_MED_U6M_U9M_CONS_LIV,VL_RAZ_MED_U9M_U12M_CONS_LIV,VL_RAZ_TOT_U3M_U6M_CONS_ROU,VL_RAZ_TOT_U6M_U9M_CONS_ROU,VL_RAZ_TOT_U9M_U12M_CONS_ROU,VL_RAZ_MED_U3M_U6M_CONS_ROU,VL_RAZ_MED_U6M_U9M_CONS_ROU,VL_RAZ_MED_U9M_U12M_CONS_ROU,PK_DAT_PROC,PK_DATREF
0,1,21,F,1331,1,Intermediário,1,2022-12-18,332,1064,...,1.0,1.05,0.97,1.00,0.5,1.46,1.00,0.66,2023-11-15,202311
1,2,21,M,1160,0,Intermediário,0,2021-12-07,708,701,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-15,202311
2,3,62,M,454,1,Básico,0,2022-11-21,359,1033,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-15,202311
3,4,64,M,226,1,Intermediário,0,2022-12-12,338,837,...,NaN,NaN,NaN,0.49,1.0,NaN,0.97,1.00,2023-11-15,202311
4,5,61,M,474,1,Avançado,0,2022-12-22,328,1064,...,NaN,NaN,1.00,1.00,1.0,1.00,1.00,1.00,2023-11-15,202311


<br>

## **Tratamento da nossa ABT de treino**

### **Validação cruzada Holdout 70/30.**
- Vamos fazer o split dos dados em 70% para treino e 30% para teste (também poderia ser 80/20). Essa separação é importante para generalização do modelo, ou seja, para que o modelo não decore os dados acarretando em overfitting, para evitar o Data Leakage (vazamento de dados) e para estar preparado para o ambiente de produção.

In [ ]:
# Dividindo o DataFrame em treino e teste.
abt_churn_01, abt_churn_01_test = train_test_split(abt_churn_00, test_size=0.3, random_state=42)

# Verificando quantidade de linhas e colunas das tabelas.
abt_churn_01.shape, abt_churn_01_test.shape

((700, 99), (300, 99))

In [ ]:
abt_churn_01.head()

,ID_CLIENTE,IDADE,NM_GENERO,QT_DIAS_DESDE_INSCR,FL_USOU_SUPORTE,PLANO,CHURN,UTL_DATA_COMPRA,DIAS_DESDE_ULT_TRANS,QTD_DIAS_DESDE_PRIM_COMPRA,...,VL_RAZ_MED_U6M_U9M_CONS_LIV,VL_RAZ_MED_U9M_U12M_CONS_LIV,VL_RAZ_TOT_U3M_U6M_CONS_ROU,VL_RAZ_TOT_U6M_U9M_CONS_ROU,VL_RAZ_TOT_U9M_U12M_CONS_ROU,VL_RAZ_MED_U3M_U6M_CONS_ROU,VL_RAZ_MED_U6M_U9M_CONS_ROU,VL_RAZ_MED_U9M_U12M_CONS_ROU,PK_DAT_PROC,PK_DATREF
541,542,67,F,997,0,Intermediário,0,2022-03-21,604,161,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-15,202311
440,441,55,F,1777,1,Intermediário,0,2022-04-15,579,812,...,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,2023-11-15,202311
482,483,62,F,1721,1,Básico,0,2022-12-28,322,1006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-15,202311
422,423,63,M,1690,0,Intermediário,1,2022-04-09,585,222,...,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,2023-11-15,202311
778,779,40,F,467,0,Avançado,1,2022-10-30,381,818,...,1.0,1.0,NaN,NaN,1.0,NaN,NaN,1.0,2023-11-15,202311


<br>

### **Separação das variáveis**
- Lista de variáveis para retirar dos tratamentos, pois não queremos normalizar ou padronizar elas.

In [ ]:
# Criando lista.
lista_spec = ['ID_CLIENTE', 'CHURN', 'PK_DAT_PROC','PK_DATREF']

abt_churn_02 = abt_churn_01.drop(axis=1,columns = lista_spec)
abt_churn_02.shape

(700, 95)

In [ ]:
abt_churn_02.head()

,IDADE,NM_GENERO,QT_DIAS_DESDE_INSCR,FL_USOU_SUPORTE,PLANO,UTL_DATA_COMPRA,DIAS_DESDE_ULT_TRANS,QTD_DIAS_DESDE_PRIM_COMPRA,QTD_COMPRAS_MES,VL_TOT_GASTO,...,VL_RAZ_TOT_U9M_U12M_CONS_LIV,VL_RAZ_MED_U3M_U6M_CONS_LIV,VL_RAZ_MED_U6M_U9M_CONS_LIV,VL_RAZ_MED_U9M_U12M_CONS_LIV,VL_RAZ_TOT_U3M_U6M_CONS_ROU,VL_RAZ_TOT_U6M_U9M_CONS_ROU,VL_RAZ_TOT_U9M_U12M_CONS_ROU,VL_RAZ_MED_U3M_U6M_CONS_ROU,VL_RAZ_MED_U6M_U9M_CONS_ROU,VL_RAZ_MED_U9M_U12M_CONS_ROU
541,67,F,997,0,Intermediário,2022-03-21,604,161,0.08,304.84,...,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
440,55,F,1777,1,Intermediário,2022-04-15,579,812,0.26,998.90,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0
482,62,F,1721,1,Básico,2022-12-28,322,1006,0.43,1508.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422,63,M,1690,0,Intermediário,2022-04-09,585,222,0.15,430.84,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0
778,40,F,467,0,Avançado,2022-10-30,381,818,0.10,364.08,...,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,NaN,1.0


<br>

### **Verificação dos Metadados**

In [ ]:
# Função para gerar os metadados.
def generate_metadata(dataframe):
    """
    Gera um dataframe contendo metadados das colunas do dataframe fornecido.
    :param dataframe: DataFrame para o qual os metadados serão gerados.
    :return: DataFrame contendo metadados.
    """

    # Coleta de metadados básicos.
    metadata = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata=metadata.sort_values(by='percent_nulos',ascending=False)
    metadata = metadata.reset_index(drop=True)

    return metadata

In [ ]:
# Obtendo os metadados referente à nossa ABT.
metadata_df = generate_metadata(abt_churn_02)
metadata_df.tail(95)

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,VL_MED_U3M_CONS_ROUPAS,float64,485,69.29,213
1,VL_RAZ_TOT_U3M_U6M_CONS_ALI,float64,485,69.29,25
2,VL_RAZ_MED_U3M_U6M_CONS_ROU,float64,485,69.29,33
3,VL_TOT_U3M_CONS_ROUPAS,float64,485,69.29,214
4,VL_MED_U3M_CONS_ALIMENTOS,float64,485,69.29,214
...,...,...,...,...,...
90,VL_TOT_GASTO_ROUPAS,float64,0,0.00,555
91,CATEG_FAVORITA,object,0,0.00,5
92,QTD_CATEG_DIF,int64,0,0.00,5
93,NM_GENERO,object,0,0.00,2


<br>

### **Tratamento de valores nulos**

**Excluir variáveis com alta porcentagem de valores nulos.**
- Geralmente excluimos somente variáveis com mais de 80% de valores nulos, porém em nossa ABT não há nenhuma variável com mais de 80%, sendo assim, como o objetivo desse projeto é reforçar na prática os estudos teóricos, vou colocar como limite aceitável 69% de valores nulos para manter a variável, e todas as variáveis que tiverem mais de 69% serão excluídas.

In [ ]:
# Determinando o limite de valores nulos, neste caso até 69%.
missing_cutoff = 69

# Definindo as variáveis com valores nulos acima de 69% e gravando ela em uma lista.
df_drop_nulos = metadata_df[(metadata_df['percent_nulos'] > missing_cutoff)]
lista_drop_nulos = list(df_drop_nulos.nome_variavel.values)
print('Variáveis que serão excluídas por alto percentual de nulos:', lista_drop_nulos)

# Retirando a lista de variáveis com alto percentual de nulos.
abt_churn_03 = abt_churn_02.drop(axis=1,columns=lista_drop_nulos)

abt_churn_03.shape

Variáveis que serão excluídas por alto percentual de nulos: ['VL_MED_U3M_CONS_ROUPAS', 'VL_RAZ_TOT_U3M_U6M_CONS_ALI', 'VL_RAZ_MED_U3M_U6M_CONS_ROU', 'VL_TOT_U3M_CONS_ROUPAS', 'VL_MED_U3M_CONS_ALIMENTOS', 'VL_RAZ_TOT_U3M_U6M_CONS_ROU', 'VL_RAZ_MED_U3M_U6M_CONS_ALI', 'VL_TOT_U3M_CONS_ALIMENTOS']


(700, 87)

In [ ]:
# Salvando a lista de variáveis dropadas em um arquivo .pkl (artefato).
with open('/content/drive/MyDrive/artefatos/prd_drop_nullvars.pkl', 'wb') as f:
    pickle.dump(lista_drop_nulos, f)

In [ ]:
abt_churn_03.head()

,IDADE,NM_GENERO,QT_DIAS_DESDE_INSCR,FL_USOU_SUPORTE,PLANO,UTL_DATA_COMPRA,DIAS_DESDE_ULT_TRANS,QTD_DIAS_DESDE_PRIM_COMPRA,QTD_COMPRAS_MES,VL_TOT_GASTO,...,VL_RAZ_TOT_U3M_U6M_CONS_LIV,VL_RAZ_TOT_U6M_U9M_CONS_LIV,VL_RAZ_TOT_U9M_U12M_CONS_LIV,VL_RAZ_MED_U3M_U6M_CONS_LIV,VL_RAZ_MED_U6M_U9M_CONS_LIV,VL_RAZ_MED_U9M_U12M_CONS_LIV,VL_RAZ_TOT_U6M_U9M_CONS_ROU,VL_RAZ_TOT_U9M_U12M_CONS_ROU,VL_RAZ_MED_U6M_U9M_CONS_ROU,VL_RAZ_MED_U9M_U12M_CONS_ROU
541,67,F,997,0,Intermediário,2022-03-21,604,161,0.08,304.84,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
440,55,F,1777,1,Intermediário,2022-04-15,579,812,0.26,998.90,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0
482,62,F,1721,1,Básico,2022-12-28,322,1006,0.43,1508.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422,63,M,1690,0,Intermediário,2022-04-09,585,222,0.15,430.84,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0
778,40,F,467,0,Avançado,2022-10-30,381,818,0.10,364.08,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,1.0


**Substituir os valores nulos.**
- Pela média para variáveis numéricas.
- Por "MISS_VERIFICAR" para categóricas.

In [ ]:
# Substituindo nulos pela média para as variáveis numéricas.
abt_churn_04 = abt_churn_03.copy()
means = abt_churn_04.select_dtypes(include=['float64', 'int64', 'int32']).mean()
abt_churn_04.fillna(means, inplace=True)

# Salvando a substituição de nulos (numéricas) em um arquivo .pkl (artefato).
with open('/content/drive/MyDrive/artefatos/prd_fillna_num.pkl', 'wb') as f:
  pickle.dump(means, f)

abt_churn_04.shape

(700, 87)

In [ ]:
# Substituindo nulos por "MISS_VERIFICAR" para as variáveis categóricas.
abt_churn_05 = abt_churn_04.copy()
categorical_cols = abt_churn_05.select_dtypes(include=['object']).columns
abt_churn_05[categorical_cols] = abt_churn_05[categorical_cols].fillna('MISS_VERIFICAR')

abt_churn_05.shape

(700, 87)

In [ ]:
abt_churn_05.head()

,IDADE,NM_GENERO,QT_DIAS_DESDE_INSCR,FL_USOU_SUPORTE,PLANO,UTL_DATA_COMPRA,DIAS_DESDE_ULT_TRANS,QTD_DIAS_DESDE_PRIM_COMPRA,QTD_COMPRAS_MES,VL_TOT_GASTO,...,VL_RAZ_TOT_U3M_U6M_CONS_LIV,VL_RAZ_TOT_U6M_U9M_CONS_LIV,VL_RAZ_TOT_U9M_U12M_CONS_LIV,VL_RAZ_MED_U3M_U6M_CONS_LIV,VL_RAZ_MED_U6M_U9M_CONS_LIV,VL_RAZ_MED_U9M_U12M_CONS_LIV,VL_RAZ_TOT_U6M_U9M_CONS_ROU,VL_RAZ_TOT_U9M_U12M_CONS_ROU,VL_RAZ_MED_U6M_U9M_CONS_ROU,VL_RAZ_MED_U9M_U12M_CONS_ROU
541,67,F,997,0,Intermediário,2022-03-21,604,161,0.08,304.84,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.925793,0.921579,0.992362,1.009152
440,55,F,1777,1,Intermediário,2022-04-15,579,812,0.26,998.90,...,0.918065,0.909307,0.910273,0.984424,1.005693,0.998424,1.000000,1.000000,1.000000,1.000000
482,62,F,1721,1,Básico,2022-12-28,322,1006,0.43,1508.11,...,0.918065,0.909307,0.910273,0.984424,1.005693,0.998424,0.925793,0.921579,0.992362,1.009152
422,63,M,1690,0,Intermediário,2022-04-09,585,222,0.15,430.84,...,0.918065,0.909307,0.910273,0.984424,1.005693,0.998424,1.000000,1.000000,1.000000,1.000000
778,40,F,467,0,Avançado,2022-10-30,381,818,0.10,364.08,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.925793,1.000000,0.992362,1.000000


In [ ]:
# Obtendo os metadados referente à nossa ABT.
metadata_df_02 = generate_metadata(abt_churn_05)
metadata_df_02.head(95)

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,IDADE,int32,0,0.0,52
1,VL_TOT_U6M_CONS_ROUPAS,float64,0,0.0,271
2,VL_RAZ_MED_U6M_U9M_CONS_ALI,float64,0,0.0,36
3,VL_RAZ_TOT_U9M_U12M_CONS_ALI,float64,0,0.0,46
4,VL_RAZ_TOT_U6M_U9M_CONS_ALI,float64,0,0.0,35
...,...,...,...,...,...
82,VL_TOT_U12M_CONS_ALIMENTOS,float64,0,0.0,385
83,VL_TOT_U9M_CONS_ALIMENTOS,float64,0,0.0,340
84,VL_TOT_U6M_CONS_ALIMENTOS,float64,0,0.0,295
85,QTD_CATEG_DIF,int64,0,0.0,5


<br>

### **Tratamento de variáveis categóricas de alta cardinalidade (Label Encoder)**

In [ ]:
# Definindo a cardinalidade mínima, neste caso será 20.
card_cutoff = 20

# Verificando as variáveis com alta cardinalidade e salvado elas em uma lista.
df_categ_labelenc = metadata_df[(metadata_df['cardinalidade'] > card_cutoff) & (metadata_df['tipo'] == 'object')]
lista_labelenc = list(df_categ_labelenc.nome_variavel.values)

# Iterando sobre as listas criadas.
for item in lista_drop_nulos:
    if item in lista_labelenc:
        lista_labelenc.remove(item)

print('Lista de variáveis para Label Encoding:', lista_labelenc)

Lista de variáveis para Label Encoding: ['UTL_DATA_COMPRA']


In [ ]:
# Criando um dicionário vazio.
encoders = {}

# Iterando para o encoding.
for col in lista_labelenc:
    encoder = LabelEncoder()
    abt_churn_05[col] = encoder.fit_transform(abt_churn_05[col])

    # Armazenando o encoder para a coluna atual em um dicionário.
    encoders[col] = encoder

# Salvando o dicionário de encoders e a lista de colunas em um arquivo .pkl.
data_to_serialize = {
    'encoders': encoders,
    'columns': lista_labelenc
}

with open('/content/drive/MyDrive/artefatos/prd_labelenc.pkl', 'wb') as f:
    pickle.dump(data_to_serialize, f)

In [ ]:
abt_churn_05.head()

,IDADE,NM_GENERO,QT_DIAS_DESDE_INSCR,FL_USOU_SUPORTE,PLANO,UTL_DATA_COMPRA,DIAS_DESDE_ULT_TRANS,QTD_DIAS_DESDE_PRIM_COMPRA,QTD_COMPRAS_MES,VL_TOT_GASTO,...,VL_RAZ_TOT_U3M_U6M_CONS_LIV,VL_RAZ_TOT_U6M_U9M_CONS_LIV,VL_RAZ_TOT_U9M_U12M_CONS_LIV,VL_RAZ_MED_U3M_U6M_CONS_LIV,VL_RAZ_MED_U6M_U9M_CONS_LIV,VL_RAZ_MED_U9M_U12M_CONS_LIV,VL_RAZ_TOT_U6M_U9M_CONS_ROU,VL_RAZ_TOT_U9M_U12M_CONS_ROU,VL_RAZ_MED_U6M_U9M_CONS_ROU,VL_RAZ_MED_U9M_U12M_CONS_ROU
541,67,F,997,0,Intermediário,90,604,161,0.08,304.84,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.925793,0.921579,0.992362,1.009152
440,55,F,1777,1,Intermediário,102,579,812,0.26,998.90,...,0.918065,0.909307,0.910273,0.984424,1.005693,0.998424,1.000000,1.000000,1.000000,1.000000
482,62,F,1721,1,Básico,308,322,1006,0.43,1508.11,...,0.918065,0.909307,0.910273,0.984424,1.005693,0.998424,0.925793,0.921579,0.992362,1.009152
422,63,M,1690,0,Intermediário,99,585,222,0.15,430.84,...,0.918065,0.909307,0.910273,0.984424,1.005693,0.998424,1.000000,1.000000,1.000000,1.000000
778,40,F,467,0,Avançado,251,381,818,0.10,364.08,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.925793,1.000000,0.992362,1.000000


<br>

### **Tratamento de variáveis categóricas de baixa cardinalidade (OneHot Encoder)**

In [ ]:
# Definindo a cardinalidade máxima.
card_cutoff = 20

# Verificando as variáveis com baixa cardinalidade e salvado elas em uma lista.
df_categ_onehot = metadata_df[(metadata_df['cardinalidade'] <= card_cutoff) & (metadata_df['tipo'] == 'object')]
lista_onehot = list(df_categ_onehot.nome_variavel.values)

# Iterando sobre as listas criadas.
for item in lista_drop_nulos:
    if item in lista_onehot:
        lista_onehot.remove(item)

print('Lista de variáveis para OneHot Encoding:', lista_onehot)

Lista de variáveis para OneHot Encoding: ['PLANO', 'CATEG_FAVORITA', 'NM_GENERO']


In [ ]:
# Instanciando o encoder.
encoder2 = OneHotEncoder(drop='first', sparse_output=False)

# Aplicando o onehot encoding.
encoded_data = encoder2.fit_transform(abt_churn_05[lista_onehot])
encoded_cols = encoder2.get_feature_names_out(lista_onehot)
encoded_df = pd.DataFrame(encoded_data, columns=encoded_cols, index=abt_churn_05.index)

abt_churn_05 = pd.concat([abt_churn_05.drop(lista_onehot, axis=1), encoded_df], axis=1)

# Salvando o encoder e a lista de colunas em um arquivo .pkl.
data_to_serialize2 = {
    'encoder': encoder2,
    'columns': lista_onehot
}

with open('/content/drive/MyDrive/artefatos/prd_onehotenc.pkl', 'wb') as f:
    pickle.dump(data_to_serialize2, f)

In [ ]:
abt_churn_05.head()

,IDADE,QT_DIAS_DESDE_INSCR,FL_USOU_SUPORTE,UTL_DATA_COMPRA,DIAS_DESDE_ULT_TRANS,QTD_DIAS_DESDE_PRIM_COMPRA,QTD_COMPRAS_MES,VL_TOT_GASTO,VL_MED_GASTO,VL_MAX_GASTO,...,VL_RAZ_TOT_U9M_U12M_CONS_ROU,VL_RAZ_MED_U6M_U9M_CONS_ROU,VL_RAZ_MED_U9M_U12M_CONS_ROU,PLANO_Básico,PLANO_Intermediário,CATEG_FAVORITA_Eletrônicos,CATEG_FAVORITA_Esportes,CATEG_FAVORITA_Livros,CATEG_FAVORITA_Roupas,NM_GENERO_M
541,67,997,0,90,604,161,0.08,304.84,152.42,158.61,...,0.921579,0.992362,1.009152,0.0,1.0,1.0,0.0,0.0,0.0,0.0
440,55,1777,1,102,579,812,0.26,998.90,83.24,190.55,...,1.000000,1.000000,1.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0
482,62,1721,1,308,322,1006,0.43,1508.11,79.37,176.20,...,0.921579,0.992362,1.009152,1.0,0.0,0.0,1.0,0.0,0.0,0.0
422,63,1690,0,99,585,222,0.15,430.84,107.71,143.14,...,1.000000,1.000000,1.000000,0.0,1.0,1.0,0.0,0.0,0.0,1.0
778,40,467,0,251,381,818,0.10,364.08,91.02,189.14,...,1.000000,0.992362,1.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0


<br>

### **Tratamento de variáveis numéricas**
- Para fins de estudos iremos aplicar tanto a Normalização quanto a Padronização nos dados (em dataframes diferentes), para colocar em práticas as duas técnicas, porém em um ambiente real precisamos escolher uma ou outra.

- O correto seria fazer testes com os dois tratamentos e ver qual apresenta o melhor resultado, mas como o intuito aqui é fazer somente o DataPrep, sem treinar o modelo por enquanto, vamos seguir com os dados padronizados, pois a padronização leva vantagem em dados com distribuição normal, não sabemos ainda se esse é o caso dos nossos dados, mas se for terá uma vantagem.

**Normalização**

In [ ]:
# Fazendo uma cópia da ABT.
abt_churn_06 = abt_churn_05.copy()

# Instanciando o scaler.
minmax_scaler = MinMaxScaler()

# Selecionando colunas numéricas.
numeric_cols = abt_churn_06.select_dtypes(include=['float64', 'int64','int32']).columns

# Aplicando a normalização.
abt_churn_06[numeric_cols] = minmax_scaler.fit_transform(abt_churn_06[numeric_cols])

# Salvando o scaler em um arquivo .pkl.
with open('/content/drive/MyDrive/artefatos/prd_minmax_scaler.pkl', 'wb') as f:
    pickle.dump(minmax_scaler, f)

abt_churn_06.shape

(700, 91)

**Padronização**

In [ ]:
# Fazendo uma cópia da ABT.
abt_churn_07 = abt_churn_05.copy()

# Instanciando o scaler.
std_scaler = StandardScaler()

# Selecionando colunas numéricas.
numeric_cols = abt_churn_07.select_dtypes(include=['float64', 'int64','int32']).columns

# Aplicando a padronização.
abt_churn_07[numeric_cols] = std_scaler.fit_transform(abt_churn_07[numeric_cols])

# Salvando o scaler em um arquivo .pkl.
with open('/content/drive/MyDrive/artefatos/prd_std_scaler.pkl', 'wb') as f:
    pickle.dump(std_scaler, f)

abt_churn_07.shape

(700, 91)

In [ ]:
abt_churn_07.head()

,IDADE,QT_DIAS_DESDE_INSCR,FL_USOU_SUPORTE,UTL_DATA_COMPRA,DIAS_DESDE_ULT_TRANS,QTD_DIAS_DESDE_PRIM_COMPRA,QTD_COMPRAS_MES,VL_TOT_GASTO,VL_MED_GASTO,VL_MAX_GASTO,...,VL_RAZ_TOT_U9M_U12M_CONS_ROU,VL_RAZ_MED_U6M_U9M_CONS_ROU,VL_RAZ_MED_U9M_U12M_CONS_ROU,PLANO_Básico,PLANO_Intermediário,CATEG_FAVORITA_Eletrônicos,CATEG_FAVORITA_Esportes,CATEG_FAVORITA_Livros,CATEG_FAVORITA_Roupas,NM_GENERO_M
541,1.583371,0.135225,-1.049811,-1.323513,0.723643,-2.386224,-1.299335,-1.245930,2.150143,-0.548968,...,8.082423e-16,-1.118994e-15,-1.681573e-15,-0.699544,1.367060,1.898753,-0.524448,-0.531085,-0.431788,-1.017292
440,0.785320,1.646397,0.952553,-1.185050,0.588833,0.029952,0.243120,-0.060821,-0.875874,0.515401,...,5.709052e-01,7.698724e-02,-6.930967e-02,-0.699544,1.367060,-0.526661,-0.524448,-0.531085,-0.431788,-1.017292
482,1.250849,1.537903,0.952553,1.191907,-0.797010,0.749980,1.699884,0.808657,-1.045152,0.037201,...,8.082423e-16,-1.118994e-15,-1.681573e-15,1.429503,-0.731497,-0.526661,1.906767,-0.531085,-0.431788,-1.017292
422,1.317354,1.477843,-1.049811,-1.219666,0.621187,-2.159824,-0.699491,-1.030785,0.194473,-1.064490,...,5.709052e-01,7.698724e-02,-6.930967e-02,-0.699544,1.367060,1.898753,-0.524448,-0.531085,-0.431788,0.983002
778,-0.212243,-0.891597,-1.049811,0.534205,-0.478859,0.052221,-1.127951,-1.144778,-0.535567,0.468414,...,5.709052e-01,-1.118994e-15,-6.930967e-02,-0.699544,-0.731497,-0.526661,-0.524448,1.882938,-0.431788,-1.017292


<br>

### **Trazer o ID e Target para a tabela pós DataPrep**

In [ ]:
# Fazendo o merge das colunas para a ABT tratada.
abt_churn_tratada = pd.merge(abt_churn_07, abt_churn_01[['ID_CLIENTE', 'CHURN']], left_index=True, right_index=True, how='inner')

abt_churn_tratada.shape

(700, 93)

In [ ]:
abt_churn_tratada.head()

,IDADE,QT_DIAS_DESDE_INSCR,FL_USOU_SUPORTE,UTL_DATA_COMPRA,DIAS_DESDE_ULT_TRANS,QTD_DIAS_DESDE_PRIM_COMPRA,QTD_COMPRAS_MES,VL_TOT_GASTO,VL_MED_GASTO,VL_MAX_GASTO,...,VL_RAZ_MED_U9M_U12M_CONS_ROU,PLANO_Básico,PLANO_Intermediário,CATEG_FAVORITA_Eletrônicos,CATEG_FAVORITA_Esportes,CATEG_FAVORITA_Livros,CATEG_FAVORITA_Roupas,NM_GENERO_M,ID_CLIENTE,CHURN
541,1.583371,0.135225,-1.049811,-1.323513,0.723643,-2.386224,-1.299335,-1.245930,2.150143,-0.548968,...,-1.681573e-15,-0.699544,1.367060,1.898753,-0.524448,-0.531085,-0.431788,-1.017292,542,0
440,0.785320,1.646397,0.952553,-1.185050,0.588833,0.029952,0.243120,-0.060821,-0.875874,0.515401,...,-6.930967e-02,-0.699544,1.367060,-0.526661,-0.524448,-0.531085,-0.431788,-1.017292,441,0
482,1.250849,1.537903,0.952553,1.191907,-0.797010,0.749980,1.699884,0.808657,-1.045152,0.037201,...,-1.681573e-15,1.429503,-0.731497,-0.526661,1.906767,-0.531085,-0.431788,-1.017292,483,0
422,1.317354,1.477843,-1.049811,-1.219666,0.621187,-2.159824,-0.699491,-1.030785,0.194473,-1.064490,...,-6.930967e-02,-0.699544,1.367060,1.898753,-0.524448,-0.531085,-0.431788,0.983002,423,1
778,-0.212243,-0.891597,-1.049811,0.534205,-0.478859,0.052221,-1.127951,-1.144778,-0.535567,0.468414,...,-6.930967e-02,-0.699544,-0.731497,-0.526661,-0.524448,1.882938,-0.431788,-1.017292,779,1


<br>

## **Fazer os mesmos tratamentos para a tabela de teste**
- Para fins de estudos, irei fazer nesta etapa o processo de tratamento dos dados de teste (dados novos), usando os artefatos salvos anteriormente no tratamento da tabela de treino.

**Separação das variáveis**

In [ ]:
# Separando lista já criada.
abt_churn_tratada_test = abt_churn_01_test.drop(axis=1,columns = lista_spec)
abt_churn_tratada_test.shape

(300, 95)

**Excluir variáveis com alta porcentagem de valores nulos**

In [ ]:
# Carregando o pkl com a lista de variáveis a serem removidas.
with open('/content/drive/MyDrive/artefatos/prd_drop_nullvars.pkl', 'rb') as f:
    loaded_pkl_drop_nullvars = pickle.load(f)

# Aplicando a remoção das variáveis na tabela de teste.
abt_churn_tratada_test = abt_churn_tratada_test.drop(axis=1, columns=loaded_pkl_drop_nullvars)

abt_churn_tratada_test.shape

(300, 87)

**Substituir os valores nulos (numéricas)**

In [ ]:
# Carregando o pkl com a substituição de nulos pelas médias.
with open('/content/drive/MyDrive/artefatos/prd_fillna_num.pkl', 'rb') as f:
    loaded_pkl_fillna_num = pickle.load(f)

# Aplicando a substituição de nulos nas variáveis numéricas na tabela de teste.
for col, mean_value in loaded_pkl_fillna_num.items():
  abt_churn_tratada_test[col].fillna(mean_value, inplace=True)

abt_churn_tratada_test.shape

(300, 87)

**Substituir os valores nulos (categóricas)**

In [ ]:
# Substituindo nulos por "MISS_VERIFICAR" para as variáveis categóricas.
categorical_cols_test = abt_churn_tratada_test.select_dtypes(include=['object']).columns
abt_churn_tratada_test[categorical_cols_test] = abt_churn_tratada_test[categorical_cols_test].fillna('MISS_VERIFICAR')

abt_churn_tratada_test.shape

(300, 87)

In [ ]:
# Verificando se ainda existem valores nulos.
abt_churn_tratada_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 521 to 155
Data columns (total 87 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   IDADE                         300 non-null    int32  
 1   NM_GENERO                     300 non-null    object 
 2   QT_DIAS_DESDE_INSCR           300 non-null    int32  
 3   FL_USOU_SUPORTE               300 non-null    int32  
 4   PLANO                         300 non-null    object 
 5   UTL_DATA_COMPRA               300 non-null    object 
 6   DIAS_DESDE_ULT_TRANS          300 non-null    int32  
 7   QTD_DIAS_DESDE_PRIM_COMPRA    300 non-null    int32  
 8   QTD_COMPRAS_MES               300 non-null    float64
 9   VL_TOT_GASTO                  300 non-null    float64
 10  VL_MED_GASTO                  300 non-null    float64
 11  VL_MAX_GASTO                  300 non-null    float64
 12  VL_MIN_GASTO                  300 non-null    float64
 13  VL_

**Tratamento de variáveis categóricas de alta cardinalidade (Label Encoder)**

In [ ]:
# Carregando o pkl com o dicionário de encoders e a lista de colunas.
with open('/content/drive/MyDrive/artefatos/prd_labelenc.pkl', 'rb') as f:
    loaded_pkl_labelenc = pickle.load(f)

loaded_encoders = loaded_pkl_labelenc['encoders']
loaded_columns = loaded_pkl_labelenc['columns']

# Aplicando o Label Encoding na tabela de teste.
for col in loaded_columns:
    if col in loaded_encoders:
        # Lidar com valores desconhecidos durante a transformação.
        abt_churn_tratada_test[col] = abt_churn_tratada_test[col].apply(lambda x: loaded_encoders[col].transform([x])[0] if x in loaded_encoders[col].classes_ else -1)

abt_churn_tratada_test.shape

(300, 87)

**Tratamento de variáveis categóricas de baixa cardinalidade (OneHot Encoder)**

In [ ]:
# Carregando o pkl com o dicionário de encoders e a lista de colunas.
with open('/content/drive/MyDrive/artefatos/prd_onehotenc.pkl', 'rb') as f:
    loaded_pkl_onehotenc = pickle.load(f)

loaded_encoders2 = loaded_pkl_onehotenc['encoder']
loaded_columns2 = loaded_pkl_onehotenc['columns']

# Aplicando o OneHot Encoding na tabela de teste.
encoded_data_test = loaded_encoders2.transform(abt_churn_tratada_test[loaded_columns2])
encoded_cols_test = loaded_encoders2.get_feature_names_out(loaded_columns2)
encoded_df_test = pd.DataFrame(encoded_data_test, columns=encoded_cols_test, index=abt_churn_tratada_test.index)

abt_churn_tratada_test = pd.concat([abt_churn_tratada_test.drop(loaded_columns2, axis=1), encoded_df_test], axis=1)

abt_churn_tratada_test.shape

(300, 91)

**Tratamento de variáveis numéricas**

In [ ]:
# Carregando o pkl com o scaler.
with open('/content/drive/MyDrive/artefatos/prd_std_scaler.pkl', 'rb') as f:
    loaded_pkl_std_scaler = pickle.load(f)

# Selecionando as colunas numéricas na tabela de teste.
numeric_cols_test = abt_churn_tratada_test.select_dtypes(include=['float64', 'int64', 'int32']).columns

# Aplicando a normalização usando o scaler carregado.
abt_churn_tratada_test[numeric_cols_test] = loaded_pkl_std_scaler.transform(abt_churn_tratada_test[numeric_cols_test])

abt_churn_tratada_test.shape

(300, 91)

**Trazer o ID e Target para a tabela pós DataPrep**

In [ ]:
# Fazendo o merge das colunas para a ABT tratada.
abt_churn_tratada_test = pd.merge(abt_churn_tratada_test, abt_churn_01_test[['ID_CLIENTE', 'CHURN']], left_index=True, right_index=True, how='inner')

abt_churn_tratada_test.shape

(300, 93)

In [ ]:
abt_churn_tratada_test.head()

,IDADE,QT_DIAS_DESDE_INSCR,FL_USOU_SUPORTE,UTL_DATA_COMPRA,DIAS_DESDE_ULT_TRANS,QTD_DIAS_DESDE_PRIM_COMPRA,QTD_COMPRAS_MES,VL_TOT_GASTO,VL_MED_GASTO,VL_MAX_GASTO,...,VL_RAZ_MED_U9M_U12M_CONS_ROU,PLANO_Básico,PLANO_Intermediário,CATEG_FAVORITA_Eletrônicos,CATEG_FAVORITA_Esportes,CATEG_FAVORITA_Livros,CATEG_FAVORITA_Roupas,NM_GENERO_M,ID_CLIENTE,CHURN
521,-0.212243,-0.405310,0.952553,-0.296576,-0.052861,0.430792,1.185732,0.967182,-0.397783,0.541060,...,-3.722350e-01,-0.699544,-0.731497,-0.526661,-0.524448,-0.531085,2.315953,-1.017292,522,1
737,1.250849,-1.308138,0.952553,1.145753,-0.775440,1.032052,1.100040,1.142593,-0.133148,0.762998,...,-6.930967e-02,-0.699544,-0.731497,-0.526661,-0.524448,-0.531085,-0.431788,0.983002,738,0
740,0.053773,-1.155083,0.952553,1.214984,-0.807795,0.783383,1.100040,0.844395,-0.336545,-0.444664,...,-1.681573e-15,1.429503,-0.731497,-0.526661,-0.524448,1.882938,-0.431788,-1.017292,741,0
660,0.386295,-0.219319,-1.049811,0.430358,-0.430328,-2.983773,-1.299335,-1.710354,-3.079993,-4.739795,...,-1.681573e-15,-0.699544,-0.731497,1.898753,-0.524448,-0.531085,-0.431788,-1.017292,661,1
411,-0.012731,-0.060452,0.952553,0.857287,-0.640631,0.323159,1.100040,0.754956,-0.211008,0.472080,...,-6.930967e-02,-0.699544,-0.731497,-0.526661,-0.524448,-0.531085,-0.431788,0.983002,412,1


<br>

## **Salvar as tabelas de treino e teste pós DataPrep**

In [ ]:
abt_churn_tratada.to_csv('/content/drive/MyDrive/abt_churn_tratada.csv')
abt_churn_tratada_test.to_csv('/content/drive/MyDrive/abt_churn_tratada_test.csv')